# UTS Mata Kuliah Machine Learning
Nama: Arif Rhizky Gilang Purnama 

NPM: 140810170030 

Kelas: B 

Sumber Dataset: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer

In [1]:
from random import random

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv("breast-cancer-2.csv") 
df.head()

In [3]:
df.tail()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   prediction   286 non-null    object
 1   age          286 non-null    object
 2   menopause    286 non-null    object
 3   tumor_size   286 non-null    object
 4   inv_nodes    286 non-null    object
 5   node_caps    286 non-null    object
 6   deg_malig    286 non-null    object
 7   breast       286 non-null    object
 8   breast_quad  286 non-null    object
 9   irradiat     286 non-null    object
dtypes: object(10)
memory usage: 22.5+ KB


In [4]:
df_features = df.loc[:,df.columns!='prediction']
df_target = df[['prediction']]

In [5]:
df_features_onehot = pd.get_dummies(df_features)

In [6]:
le = LabelEncoder()
labeled_features_onehot = df_features_onehot.apply(le.fit_transform)
labeled_target = df_target.apply(le.fit_transform)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(labeled_features_onehot, labeled_target, test_size=0.3)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [8]:
pd.set_option('max_columns', None)
X_test

,age_20_29,age_30_39,age_40_49,age_50_59,age_60_69,age_70_79,menopause_ge40,menopause_lt40,menopause_premeno,tumor_size_0_4,tumor_size_10_14,tumor_size_15_19,tumor_size_20_24,tumor_size_25_29,tumor_size_30_34,tumor_size_35_39,tumor_size_40_44,tumor_size_45_49,tumor_size_50_54,tumor_size_5_9,inv_nodes_0_2,inv_nodes_12_14,inv_nodes_15_17,inv_nodes_24_26,inv_nodes_3_5,inv_nodes_6_8,inv_nodes_9_11,node_caps_no,node_caps_yes,deg_malig_one,deg_malig_three,deg_malig_two,breast_left,breast_right,breast_quad_central,breast_quad_left_low,breast_quad_left_up,breast_quad_right_low,breast_quad_right_up,irradiat_no,irradiat_yes
0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1
1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1
2,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0
3,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,0,1
4,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0
82,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0,1
83,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0
84,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1


In [9]:
pd.reset_option('max_columns')

# Perceptron

In [10]:
class Perceptron:
    # Kelas untuk perceptron dengan binary step activation function
    
    def __init__(self):
        self.w = None
        self.b = None
        
        self.best_b = None
        self.best_w = None
        self.best_pred = None
        self.best_epoch = None
        self.lowest_error = None
        
#     def perceptron(self, x):
#         return np.dot(x, self.w) + self.b
    
    def binary_step(self, x):
        return 1 if (x>=0) else 0
    
    def fit(self, X, Y, epochs=50, learning_rate=0.1, initialise=True):
        if initialise:
            self.w = [random()] * X.shape[1]
            self.b = 1
            
            self.best_b = 1
            self.best_w = [0] * X.shape[1]
            self.best_pred = [0] * X.shape[0]
            self.lowest_error = np.inf
        
        y_pred = [0] * X.shape[0] 
        current_epoch = 0
        
        while(current_epoch < epochs):
            error = 0
            
            for index, row in X.iterrows():
                pred = 0
                for i, value in enumerate(row):
                    pred += self.w[i] * value
                    
                pred += self.b
                y_pred[index] = self.binary_step(pred)
                
                if(y_pred[index] != Y.iloc[index, 0]):
                    for i, value in enumerate(row):
                        self.w[i] += learning_rate * (Y.iloc[index,0] - y_pred[index]) * value
                    
                    self.b += learning_rate * (Y.iloc[index,0] - y_pred[index])
                    error += 1
                if(index == X.shape[0]-1):
                    current_epoch += 1
            
            if(error < self.lowest_error):
                self.best_b = self.b
                self.best_w = self.w.copy()
                self.best_pred = y_pred.copy()
                self.best_epoch = current_epoch
                self.lowest_error = error
        
            print("Accuracy Epoch ", current_epoch," : ",str(round(1-error/X.shape[0],3)*100)[0:4],"%") #print akurasi setiap epochnya
        print("Training Finished!")
    
    def predict(self, X):
        y_pred = [0] * X.shape[0]
        pred = 0
        
        for index, row in X.iterrows():
            pred = 0
            for i, value in enumerate(row):
                pred += self.w[i] * value
                    
            pred += self.b
            y_pred[index] = self.binary_step(pred)
            
        return y_pred
        
    def get_stat(self):
        print("Best b: ", self.best_b, "\n")
        print("Best w: ", self.best_w, "\n")
        print("Best epoch: ", self.best_epoch, "\n")
        print("Lowest error: ", self.lowest_error)
        

In [14]:
model = Perceptron()
model.fit(X_test, y_test, learning_rate = 0.008)

Accuracy Epoch  1  :  32.6 %
Accuracy Epoch  2  :  32.6 %
Accuracy Epoch  3  :  32.6 %
Accuracy Epoch  4  :  60.5 %
Accuracy Epoch  5  :  76.7 %
Accuracy Epoch  6  :  75.6 %
Accuracy Epoch  7  :  76.7 %
Accuracy Epoch  8  :  75.6 %
Accuracy Epoch  9  :  76.7 %
Accuracy Epoch  10  :  76.7 %
Accuracy Epoch  11  :  77.9 %
Accuracy Epoch  12  :  76.7 %
Accuracy Epoch  13  :  79.1 %
Accuracy Epoch  14  :  80.2 %
Accuracy Epoch  15  :  80.2 %
Accuracy Epoch  16  :  77.9 %
Accuracy Epoch  17  :  77.9 %
Accuracy Epoch  18  :  79.1 %
Accuracy Epoch  19  :  76.7 %
Accuracy Epoch  20  :  79.1 %
Accuracy Epoch  21  :  79.1 %
Accuracy Epoch  22  :  76.7 %
Accuracy Epoch  23  :  79.1 %
Accuracy Epoch  24  :  77.9 %
Accuracy Epoch  25  :  79.1 %
Accuracy Epoch  26  :  74.4 %
Accuracy Epoch  27  :  74.4 %
Accuracy Epoch  28  :  73.3 %
Accuracy Epoch  29  :  76.7 %
Accuracy Epoch  30  :  80.2 %
Accuracy Epoch  31  :  77.9 %
Accuracy Epoch  32  :  72.1 %
Accuracy Epoch  33  :  79.1 %
Accuracy Epoch  34 

In [15]:
model.get_stat()

Best b:  -1.0320000000000016 

Best w:  [0.7921256221192998, 0.4561256221192995, 0.35212562211929943, 0.17612562211929927, 0.3441256221192994, 0.6001256221192997, -0.06387437788070083, 0.5441256221192996, -0.1358743778807009, 0.5201256221192996, 0.48012562211929954, 0.5281256221192996, 0.5681256221192996, 0.6401256221192997, 0.5921256221192996, 0.4321256221192995, 0.5361256221192996, 0.8001256221192998, 0.7921256221192998, 0.7921256221192998, 0.2961256221192994, 0.5521256221192996, 0.7921256221192998, 0.7921256221192998, 0.2961256221192994, 0.2961256221192994, 0.48812562211929955, -0.29587437788070103, -0.1518743778807009, 0.10412562211929921, 0.16012562211929926, 0.08012562211929919, -0.20787437788070096, -0.23987437788070098, 0.3441256221192994, 0.5361256221192996, 0.4001256221192995, 0.25612562211929935, 0.39212562211929947, -0.23987437788070098, -0.20787437788070096] 

Best epoch:  44 

Lowest error:  16


In [16]:
model_prediction = model.predict(X_test)

In [17]:
accuracy_score(model_prediction, y_test, normalize=True)

0.8255813953488372